#### Notebook to preprocess dummy data

In [ ]:
import geopandas as gpd
from shapely.geometry import Point, LineString
import random
import networkx as nx
import osmnx as ox

In [ ]:
# Params
nsw_lambert_crs = "EPSG:28356"
target_distance = 10000  # in meters
cable_length_unit_cost = 100  # cost per meter

#### Get data
- QGIS to create start and end point for transmission line
- QGIS to create project and NO-GO zones
- ELVIS to get DEM data
    - Use vector grid overlay in QGIS and use zonal stats to get elevation

In [ ]:
boundary_gdf = gpd.read_file("boundary.gpkg").to_crs(nsw_lambert_crs)
boundary_polygon = boundary_gdf.geometry.union_all()
exclusion_zones_gdf = gpd.read_file("exclusion_zones.gpkg").to_crs(nsw_lambert_crs)
elevation_gdf = gpd.read_file("elevation.gpkg").to_crs(nsw_lambert_crs)

#### Reading up different RRT algorithms
- https://github.com/AtsushiSakai/PythonRobotics

### Attempting simple RRT to generate networkX graph

#### Helper functions

In [ ]:
# Choose a random point from approved points
def choose_random_approved_point(approved_points):
    return random.choice(approved_points)

In [ ]:
# Find nearest node from existing tree to the ending point
def find_nearest_approved_to_ending(ending_point, approved_points):
    nearest_point = None
    min_distance = float('inf')
    for approved_point in approved_points:
        distance = ending_point.distance(approved_point)
        if distance < min_distance:
            min_distance = distance
            nearest_point = approved_point
    return nearest_point, min_distance

In [ ]:
# Randomly generate new node (p)
def generate_target_circle(closest_approved_point, target_distance):
    return closest_approved_point.buffer(target_distance)

def generage_rand_next_point(cur_point, target_circle, boundary_polygon, exclusion_zones, max_attempts=100):
    # Find intersection of target_circle and boundary polygon
    intersection_polygon = target_circle.intersection(boundary_polygon)
    # Get all coordinates on the boundary of this new polygon, and randomly pick one
    intersection_coords = list(intersection_polygon.exterior.coords)

    # Check if straight line between cur_point and attempt_point intersect with exclusion
    for attempt in range(max_attempts):
        attempt_point_coords = random.choice(intersection_coords)
        attempt_point = Point(attempt_point_coords)

        line_to_attempt = LineString([cur_point, attempt_point])

        intersects_exclusion = False
        for exclusion_zone in exclusion_zones.geometry:
            if line_to_attempt.intersects(exclusion_zone):
                intersects_exclusion = True
                break

        if not intersects_exclusion:
            return cur_point, attempt_point, "success"
        
    # If max attempts exhusted, remove current point from approved points as it hits a dead end
    return cur_point, cur_point, "dead_end"

In [ ]:
# Add successful node and edge to graph
def add_success_node_edge_to_graph(prev_n, next_n, G):
    G.add_node(next_n)
    G.add_edge(prev_n, next_n)

    # Find distance
    edge_length = prev_n.distance(next_n)

    # Find slope - use elevation data to calculate slope between prev_n and next_n
    prev_slope_point = elevation_gdf.geometry.distance(prev_n).idxmin()
    next_slope_point = elevation_gdf.geometry.distance(next_n).idxmin()
    prev_elevation = elevation_gdf.loc[prev_slope_point, 'elevation']
    next_elevation = elevation_gdf.loc[next_slope_point, 'elevation']
    elevation_diff = next_elevation - prev_elevation
    slope = elevation_diff / edge_length if edge_length != 0 else 0
    
    G[prev_n][next_n]['slope'] = slope

    # [TODO] Add cost based on slope and distance
    G[prev_n][next_n]['cost'] = slope * edge_length * cable_length_unit_cost
    return G

In [ ]:
# If the closest point in approved_points collection can be drawn as a straight line to ending point within target_distance, graph is complete
def graph_connected_check(approved_points, ending_point, target_distance, exclusion_zones):
    closest_approved_point, distance_to_closest = find_nearest_approved_to_ending(ending_point, approved_points)

    # Check if distance is within target_distance
    dist_check = False
    excl_check = False
    
    if distance_to_closest <= target_distance:
        dist_check = True
    else:
        return False
    
    # Check if closest_approved_point and ending_point doesn't intersect with exclusion zones
    line_to_ending = LineString([closest_approved_point, ending_point])
    for exclusion_zone in exclusion_zones.geometry:
        if line_to_ending.intersects(exclusion_zone):
            return False
        else: 
            excl_check = True
    
    # If both checks are true, return True
    if dist_check & excl_check:
        return True

##### Create graph

Algorithm:

1. Insert start point in the tree
2. While tree cannot connect to goal
    - Create circle from closest point (r) to destination with radius = target_distance
    - Sample random point (p) from edge of circle that is in boundary
    - Add branch from point r to p
    - If new branch intersects with obstacle then discard
3. Find the path from start to end through the tree

In [ ]:
# Load starting point, boundary, exclusion zones
starting_point_gdf = gpd.read_file("starting_point.gpkg").to_crs(nsw_lambert_crs)
ending_point_gdf = gpd.read_file("ending_point.gpkg").to_crs(nsw_lambert_crs)

In [ ]:
# Create collection of approved points, add starting point and intialize graph
approved_points = []
approved_points.append(starting_point_gdf.geometry.iloc[0])

G = nx.Graph()
G.add_node(starting_point_gdf.geometry.iloc[0])

In [ ]:
connect_graph_attempt = 0
while not graph_connected_check(approved_points, ending_point_gdf.geometry.iloc[0], target_distance, exclusion_zones_gdf):
    # Choose a random point from approved points
    cur_point = choose_random_approved_point(approved_points)
    target_c = generate_target_circle(cur_point, target_distance)

    # Generate random next point from chosen current point
    cur_point, next_point, status = generage_rand_next_point(cur_point, target_c, boundary_polygon, exclusion_zones_gdf)

    # If successful, add to approved points
    if status == "success":
        approved_points.append(next_point)
        print(f"New point added at: {next_point}")

        # Add to graph
        G = add_success_node_edge_to_graph(cur_point, next_point, G)
    else:
        print("Dead end reached, no new point added.")
    connect_graph_attempt += 1
    print(f"Graph connection attempt: {connect_graph_attempt}, Approved points count: {len(approved_points)}")

#### Lowest cost path between start and end point

In [ ]:
# Find shortest path using weighted cost in graph G
source_node = ox.get_nearest_node(G, starting_point_gdf.geometry.iloc[0])
destination_node = ox.get_nearest_node(G, ending_point_gdf.geometry.iloc[0])

shortest_path = nx.shortest_path(G, source=source_node, target=destination_node, weight='cost')
print("Shortest path found:")
for node in shortest_path:
    print(node)